<a href="https://colab.research.google.com/github/Bergerghislain/A-fine-tunning-of-Chat-gpt-based-on-user-s-feelings/blob/main/Fine_tune_a_Masked_language_model_JeudiConfession.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

keep an eye on https://colab.research.google.com/github/huggingface/notebooks/blob/master/examples/language_modeling.ipynb#scrollTo=nRZ-5v_P3l_b

If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

# Setup

In [ ]:
! pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 27.3 MB/s 
     |████████████████████████████████| 5.5 MB 59.3 MB/s 
     |████████████████████████████████| 95 kB 5.0 MB/s 
     |████████████████████████████████| 163 kB 67.9 MB/s 
     |████████████████████████████████| 115 kB 75.2 MB/s 
     |████████████████████████████████| 212 kB 60.7 MB/s 
     |████████████████████████████████| 127 kB 72.9 MB/s 
     |████████████████████████████████| 7.6 MB 54.9 MB/s 
     |████████████████████████████████| 115 kB 68.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: dill
    Found existing installation: dill 0.3.6
    Uninstalling dill-0.3.6:
      Successfully uninstalled dill-0.3.6


If you're opening this notebook locally, make sure your environment has an install from the last version of those libraries.

To be able to share your model with the community and generate results like the one shown in the picture below via the inference API, there are a few more steps to follow.

First you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then execute the following cell and input your username and password:

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 134 kB/s 


Then you need to install Git-LFS. Uncomment the following instructions:

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# !apt install git-lfs

In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

In [ ]:
import transformers

print(transformers.__version__)

4.24.0


You can find a script version of this notebook to fine-tune your model in a distributed fashion using multiple GPUs or TPUs [here](https://github.com/huggingface/transformers/tree/master/examples/language-modeling).

In this notebook, we'll see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model on a language modeling tasks. We will cover two types of language modeling tasks which are:

- Causal language modeling: the model has to predict the next token in the sentence (so the labels are the same as the inputs shifted to the right). To make sure the model does not cheat, it gets an attention mask that will prevent it to access the tokens after token i when trying to predict the token i+1 in the sentence.

![Widget inference representing the causal language modeling task](https://github.com/huggingface/notebooks/blob/master/examples/images/causal_language_modeling.png?raw=1)

- Masked language modeling: the model has to predict some tokens that are masked in the input. It still has access to the whole sentence, so it can use the tokens before and after the tokens masked to predict their value.

![Widget inference representing the masked language modeling task](https://github.com/huggingface/notebooks/blob/master/examples/images/masked_language_modeling.png?raw=1)

We will see how to easily load and preprocess the dataset for each one of those tasks, and how to use the `Trainer` API to fine-tune a model on it.

A script version of this notebook you can directly run on a distributed environment or on TPU is available in our [examples folder](https://github.com/huggingface/transformers/tree/master/examples).

In [ ]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 35.8 MB/s 
     |████████████████████████████████| 168 kB 74.6 MB/s 
     |████████████████████████████████| 182 kB 67.4 MB/s 
     |████████████████████████████████| 63 kB 1.4 MB/s 
     |████████████████████████████████| 166 kB 52.1 MB/s 
     |████████████████████████████████| 166 kB 12.7 MB/s 
     |████████████████████████████████| 162 kB 21.3 MB/s 
     |████████████████████████████████| 162 kB 55.3 MB/s 
     |████████████████████████████████| 158 kB 31.0 MB/s 
     |████████████████████████████████| 157 kB 7.2 MB/s 
     |████████████████████████████████| 157 kB 17.2 MB/s 
     |████████████████████████████████| 157 kB 37.8 MB/s 
     |████████████████████████████████| 157 kB 59.9 MB/s 
     |████████████████████████████████| 157 kB 52.4 MB/s 
     |████████████████████████████████| 157 kB 6.0 MB/s 
     |█████████████████████████████

In [ ]:
!pip install snscrape

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Preparing the dataset

In [ ]:
import pandas as pd
from transformers import AutoTokenizer

import snscrape
import sqlite3
from snscrape.modules import twitter

In [ ]:

# import pandas as pd
# from tqdm import tqdm
# from snscrape.modules import twitter
# # Setting variables to be used below
# maxTweets = 10000

# # Creating list to append tweet data to
# tweets_list1 = []

# try:
#     # Using TwitterSearchScraper to scrape data and append tweets to list
#     for i,tweet in tqdm(enumerate(twitter.TwitterSearchScraper('#jeudiconfession').get_items()),total=maxTweets):
#         if i>maxTweets:
#             break
#         tweets_list1.append([tweet.date, tweet.id, tweet.content, tweet.user.username])
# except Exception as e:
#     # Fail case handling
#     fail_save = pd.DataFrame(tweets_list1)
#     fail_save.to_csv("tweets_saved.csv")
#     raise e

# sucess_save = pd.DataFrame(tweets_list1)
# sucess_save.to_csv("tweets_saved_10k_jeudi_confession.csv")
# print("Success!")

In [ ]:
tweets = pd.read_csv("tweets_saved_10k_jeudi_confession.csv")

In [ ]:
tweets.head()

,Unnamed: 0,0,1,2,3
0,0,2022-08-13 11:08:35+00:00,1558410182308274177,@Kemajoie Elle pensait maîtriser le #jeudicon...,MekonoG
1,1,2022-08-13 04:20:02+00:00,1558307365983850496,Ah bah Fait rien il se passera rien 😮‍💨 #Jeudi...,poopeyyy00
2,2,2022-08-12 19:45:11+00:00,1558177800640921600,#JeudiConfession \n\nMon lit à la partie où on...,williameyebe1
3,3,2022-08-12 14:47:52+00:00,1558102977457655809,"Réellement, vitalement, émotionn.Elle.ment bes...",Eliethewaiting
4,4,2022-08-12 12:44:13+00:00,1558071860578979841,"Avec une fille d'ici, ça se passait super bien...",yann__dnd


In [ ]:
pip install emoji


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 240 kB 36.2 MB/s 
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234927 sha256=8344b2947dacfd3e6b7f3e9804d16e87403241e2e0d557a04f99183d852b5123
  Stored in directory: /root/.cache/pip/wheels/f3/e3/f2/1de1c2e3ed742e1df73e0f15d58864e50c7e64f607b548d6cf
Successfully built emoji


In [ ]:
import numpy as np
import re
import emoji

import html


def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

def clean_tweet(tweet):
    if type(tweet) == float:
        return ""
    temp = tweet.lower()
    temp = html.unescape(temp)
    temp = re.sub(r'http\S+', '', temp) # remove links
    temp = re.sub("@[A-Za-z0-9_]+","username", temp) # remove usernames
    temp = re.sub(r'[^ \'\nA-Za-z0-9À-ÖØ-öø-ÿ’´.]+', ' ', temp)
    #temp = re.sub("'", "", temp) # to avoid removing contractions in english

    temp = re.sub("#jeudiconfession","jeudiconfession", temp) # remove hashtags
    temp = re.sub("#[A-Za-z0-9_]+","", temp) # remove hashtags

    temp = re.sub("’|´",'\'',temp)
    #temp = re.sub('[]', ' ', temp) #
    temp = re.sub('[\.]{2,}', ' . ', temp)
    temp = re.sub('[ ]+', ' ', temp)
    temp = re.sub(":"," ", temp)
    temp = temp.split()
    #temp = [w for w in temp if not w in stopwords]
    temp = " ".join(word for word in temp)
    return emoji.replace_emoji(temp)

In [ ]:
clean_tweet("J'ai contacté    @Puretedudragon  https://t.co/c4G4eMvlmp")# pas a dormir si mon visage")

"j'ai contacté username"

In [ ]:
real_tweets = tweets[~tweets['2'].str.startswith('@')].reset_index(drop=True)

In [ ]:
print(f"tweets that are not responses represent {len(real_tweets)/len(tweets)*100}% of complete dataset")

tweets that are not responses represent 93.26067393260674% of complete dataset


we can train another model to create responses

In [ ]:
real_tweets['2']

0       Ah bah Fait rien il se passera rien 😮‍💨 #Jeudi...
1       #JeudiConfession \n\nMon lit à la partie où on...
2       Réellement, vitalement, émotionn.Elle.ment bes...
3       Avec une fille d'ici, ça se passait super bien...
4       #JeudiConfession Si je rentre dans ta maison p...
                              ...                        
9322    Ça arrive à tout le monde 😩\n#JeudiConfession ...
9323    #JeudiConfession\nJe collabore dans 2 endroits...
9324    J'adore ce genre de TL !\nY'a une de cette amb...
9325    #jeudiconfession \nUne Fin d après midi ds la ...
9326    La blondasse est morte et enterrée🪦, j'ai repr...
Name: 2, Length: 9327, dtype: object

In [ ]:
tweets_cleaned = real_tweets['2'].apply(clean_tweet)
tweets_cleaned = pd.DataFrame([t for t in tweets_cleaned if len(t.split(' '))>5 and len(t.split(' '))<=30 and not 'anonyme' in t])

list_tweets = tweets_cleaned[0].values

In [ ]:
len(list_tweets)/len(tweets)

0.541945805419458

In [ ]:
tweets_cleaned.head(50)

,0
0,ah bah fait rien il se passera rien jeudiconfe...
1,jeudiconfession mon lit à la partie où on accr...
2,réellement vitalement émotionn.elle.ment besoi...
3,jeudiconfession si je rentre dans ta maison pi...
4,jeudiconfession mon plus vieux est parti de la...
5,comme d'habitude jeudiconfession en retard. je...
6,quand j'étais petit je pensais que le personna...
7,j'aurais trop de confessions à faire faque c'e...
8,mon jeudiconfession j'adore ma nouvelle bio
9,l'homme volait djindjan et bissap de mère tran...


In [ ]:

# for t in list_tweets:
#   if 'étudiant' in t:
#     print(t)

In [ ]:
model_name = "benjamin/gpt2-wechsel-french"

tokenizer = AutoTokenizer.from_pretrained(model_name,verbose=False,bos_token='<|startoftext|>',
                                          eos_token='<|endoftext|>', pad_token='<|pad|>')

max_length = max([len(tokenizer.encode(t,max_length=512, truncation=True)) for t in list_tweets])
print(max_length)

Downloading:   0%|          | 0.00/250 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/855k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/514k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.47M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


69


In [ ]:
from torch.utils.data import Dataset, random_split
import torch
class TweetsDataset(Dataset):
    def __init__(self, txt_list, tokenizer, max_length):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        for txt in txt_list:
            encodings_dict = tokenizer('<|startoftext|>' + txt + '<|endoftext|>', truncation=True,
                                       max_length=max_length, padding="max_length")
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]



dataset = TweetsDataset(list_tweets, tokenizer, max_length=max_length)
train_size = int(0.9 * len(dataset))
train_dataset, val_dataset = random_split(dataset, [train_size, len(dataset) - train_size])

# Fine-tuning a language model

## Fine tuning

In [ ]:
#len(model.transformer.h)

In [ ]:
import torch
from transformers import AutoModelForMaskedLM,AutoModelWithLMHead
from transformers import Trainer, TrainingArguments

model = AutoModelWithLMHead.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))

for parameter in model.parameters():
    parameter.requires_grad = False

for i, m in enumerate(model.transformer.h):
    #Only un-freeze the last n transformer blocks
    if i >= 3:
        for parameter in m.parameters():
            parameter.requires_grad = True

for parameter in model.transformer.ln_f.parameters():
    parameter.requires_grad = True

for parameter in model.lm_head.parameters():
    parameter.requires_grad = True


/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:1136: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--benjamin--gpt2-wechsel-french/snapshots/81aa3fe79c2b6b714b1fc460b4d9609153338847/config.json
Model config GPT2Config {
  "_name_or_path": "benjamin/gpt2-wechsel-french",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
 

In [ ]:


training_args = TrainingArguments(output_dir='./results',
                                  #evaluation_strategy='steps',
                                  num_train_epochs=3, logging_steps=100, save_steps=5000,
                                  per_device_train_batch_size=20, per_device_eval_batch_size=10,
                                      #eval_accumulation_steps=10,
                                  #load_best_model_at_end=True,
                                  warmup_steps=10, weight_decay=0.05, logging_dir='./logs', report_to="wandb")


PyTorch: setting up devices


In [ ]:
from evaluate import load
#perplexity = load("perplexity", module_type="metric")


In [ ]:
def compute_metrics(eval_preds):
    metric = load("perplexity", module_type="metric")
    return metric.compute(predictions=eval_preds, model_id='gpt2')


In [ ]:
trainer = Trainer(model=model,  args=training_args, train_dataset=train_dataset,
        eval_dataset=val_dataset, data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
                                                              'attention_mask': torch.stack([f[1] for f in data]),
                                                              'labels': torch.stack([f[0] for f in data])},
                  #compute_metrics=compute_metrics
                  )

In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 4878
  Num Epochs = 3
  Instantaneous batch size per device = 20
  Total train batch size (w. parallel, distributed & accumulation) = 20
  Gradient Accumulation steps = 1
  Total optimization steps = 732
  Number of trainable parameters = 102391296
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Step,Training Loss
100,20.017100
200,1.692900
300,1.488200
400,1.342300
500,1.327600
600,1.219500
700,1.208200




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=732, training_loss=3.919331060732649, metrics={'train_runtime': 267.5649, 'train_samples_per_second': 54.693, 'train_steps_per_second': 2.736, 'total_flos': 515309734656000.0, 'train_loss': 3.919331060732649, 'epoch': 3.0})

## Inference

In [ ]:
import random

In [ ]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'


In [ ]:
#help(model.generate)

In [ ]:
import torch
from transformers import AutoModelForMaskedLM,AutoModelWithLMHead
from transformers import Trainer, TrainingArguments


input_ids = tokenizer.encode("<|startoftext|>", return_tensors='pt').to(model.device)

temperature=0.7

sample_outputs = model.generate(
    input_ids,
     do_sample=True,
    num_beams = 4,
    max_length= 30,
    top_k=12,
    top_p=0.70,
    #temperature = 0.5,
    no_repeat_ngram_size=10,
    num_return_sequences=10
)
print("")
for i,tweet in enumerate(tokenizer.batch_decode(sample_outputs, skip_special_tokens=True)):
  print(bcolors.OKBLUE+"Generated tweet",i+1,":"+bcolors.ENDC)
  #print(tweet)
  tweet = tweet.strip().replace("jeudiconfession","#jeudiconfession")#write with hashtag form
  tweet = tweet.replace("username","@username")
  tweet = tweet.split(".")[0]+"."
  if 'jeudiconfession' not in tweet:
    tweet = tweet + " #jeudiconfession"
  print(f"{tweet}",end='\n'*3)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Generated tweet 1 :
#jeudiconfession j'ai un faible pour les femmes enceintes.


Generated tweet 2 :
#jeudiconfession j'ai un faible pour les hommes.


Generated tweet 3 :
#jeudiconfession j'ai un faible pour les femmes.


Generated tweet 4 :
#jeudiconfession j'ai déjà fait caca dans la cuvette de la salle de bain.


Generated tweet 5 :
. #jeudiconfession


Generated tweet 6 :
#jeudiconfession je n'aime pas les menteurs.


Generated tweet 7 :
#jeudiconfession je n'ai jamais eu de relations avec une fille.


Generated tweet 8 :
#jeudiconfession j'ai un faible pour les femmes.


Generated tweet 9 :
#jeudiconfession je n'ai jamais regardé un seul épisode d'une série télévisée.


Generated tweet 10 :
#jeudiconfession je n'ai jamais regardé la télé.


